## Identify customer segments for online retail with the use of K-means clustering


## steps that performed
Data Preparation
1. Load the Data:
 -Read data from various sources such as CSV files,
 -Excel files, and SQL databases.

3. Understand the Data:

-Explore basic information about the data, including head, info, and describe.
-Check for missing values in the dataset.

3. Clean the Data:
-Handle missing values by either dropping or filling them.
-Remove duplicate entries in the dataset.
-Correct data types as necessary.

4. Transform the Data:

-Normalize or standardize the data.
-Encode categorical variables.

5. Feature Engineering:

-Create new features based on existing ones.
-Perform feature selection to identify the most relevant features.

In [155]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")  # ignore all warnings


In [157]:
df = pd.read_csv(r'H:\Machine Learing\k-means clustring\Online Retail.csv')

In [158]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [161]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [163]:
# Checking the percentage of missing values
round(100*(df.isnull().sum()/len(df.index)), 2)

InvoiceNo       0.00
StockCode       0.00
Description     0.27
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     24.93
Country         0.00
dtype: float64

In [165]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Extract useful date parts
df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceDayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # Monday=0, Sunday=6
df['IsWeekend'] = df['InvoiceDayOfWeek'].isin([5, 6]).astype(int)

# (Optional) If you want recency features
first_date = df['InvoiceDate'].min()
df['DaysSinceFirstPurchase'] = (df['InvoiceDate'] - first_date).dt.days

# Customer-level recency (if CustomerID available)
if 'CustomerID' in df.columns:
    last_purchase = df.groupby('CustomerID')['InvoiceDate'].transform('max')
    df['DaysSinceLastPurchase'] = (last_purchase - df['InvoiceDate']).dt.days


In [167]:
# Drop rows with missing CustomerID
if 'CustomerID' in df.columns:
    missing_customers = df['CustomerID'].isnull().sum()
    print(f"Dropping {missing_customers} rows with missing CustomerID")
    df= df.dropna(subset=['CustomerID'])

Dropping 135080 rows with missing CustomerID


In [169]:
# Checking the percentage of missing values
round(100*(df.isnull().sum()/len(df.index)), 2)

InvoiceNo                 0.0
StockCode                 0.0
Description               0.0
Quantity                  0.0
InvoiceDate               0.0
UnitPrice                 0.0
CustomerID                0.0
Country                   0.0
InvoiceYear               0.0
InvoiceMonth              0.0
InvoiceDayOfWeek          0.0
IsWeekend                 0.0
DaysSinceFirstPurchase    0.0
DaysSinceLastPurchase     0.0
dtype: float64

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   InvoiceNo               406829 non-null  object        
 1   StockCode               406829 non-null  object        
 2   Description             406829 non-null  object        
 3   Quantity                406829 non-null  int64         
 4   InvoiceDate             406829 non-null  datetime64[ns]
 5   UnitPrice               406829 non-null  float64       
 6   CustomerID              406829 non-null  float64       
 7   Country                 406829 non-null  object        
 8   InvoiceYear             406829 non-null  int32         
 9   InvoiceMonth            406829 non-null  int32         
 10  InvoiceDayOfWeek        406829 non-null  int32         
 11  IsWeekend               406829 non-null  int32         
 12  DaysSinceFirstPurchase  406829 non-

In [173]:
# Count duplicate rows in whole DataFrame
df.duplicated().sum()


5225

In [175]:
# Check duplicates only on CustomerID
df.duplicated(subset=['CustomerID']).sum()

# Check duplicates on multiple columns (e.g., InvoiceNo + StockCode)
df.duplicated(subset=['InvoiceNo', 'StockCode']).sum()


10148

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   InvoiceNo               406829 non-null  object        
 1   StockCode               406829 non-null  object        
 2   Description             406829 non-null  object        
 3   Quantity                406829 non-null  int64         
 4   InvoiceDate             406829 non-null  datetime64[ns]
 5   UnitPrice               406829 non-null  float64       
 6   CustomerID              406829 non-null  float64       
 7   Country                 406829 non-null  object        
 8   InvoiceYear             406829 non-null  int32         
 9   InvoiceMonth            406829 non-null  int32         
 10  InvoiceDayOfWeek        406829 non-null  int32         
 11  IsWeekend               406829 non-null  int32         
 12  DaysSinceFirstPurchase  406829 non-

In [179]:
remaining_dup_count = df.duplicated(subset= df , keep=False).sum()
print("Remaining rows part of any duplicate group (should be 0):", remaining_dup_count)


Remaining rows part of any duplicate group (should be 0): 10062


In [181]:
before = len(df)
df.drop_duplicates(subset=['InvoiceNo', 'StockCode'], inplace=True)
after = len(df)
print("Dropped", before - after, "duplicate rows")


Dropped 10148 duplicate rows


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396681 entries, 0 to 541908
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   InvoiceNo               396681 non-null  object        
 1   StockCode               396681 non-null  object        
 2   Description             396681 non-null  object        
 3   Quantity                396681 non-null  int64         
 4   InvoiceDate             396681 non-null  datetime64[ns]
 5   UnitPrice               396681 non-null  float64       
 6   CustomerID              396681 non-null  float64       
 7   Country                 396681 non-null  object        
 8   InvoiceYear             396681 non-null  int32         
 9   InvoiceMonth            396681 non-null  int32         
 10  InvoiceDayOfWeek        396681 non-null  int32         
 11  IsWeekend               396681 non-null  int32         
 12  DaysSinceFirstPurchase  396681 non-

In [186]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,InvoiceDayOfWeek,IsWeekend,DaysSinceFirstPurchase,DaysSinceLastPurchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,2,0,0,71.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,2,0,0,71.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,2,0,0,71.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,2,0,0,71.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,2,0,0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011,12,4,0,373,0.0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011,12,4,0,373,0.0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,4,0,373,0.0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,4,0,373,0.0


In [208]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Select numeric columns
numeric_cols = ['Quantity', 'UnitPrice', 'TotalPrice', 'CustomerID_freq', 'StockCode_freq']
numeric_cols = [c for c in numeric_cols if c in df.columns]  # keep only existing

# --- Standardization ---
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])



print("Scaled numeric columns head:")
print(df[numeric_cols].head())

# Check mean and std after standardization
print("Means (should be ~0):")
print(df[numeric_cols].mean())
print("Std devs (should be ~1):")
print(df[numeric_cols].std())


Scaled numeric columns head:
   Quantity  UnitPrice  TotalPrice
0 -0.024932  -0.012968    0.026249
1 -0.024932  -0.000925    0.021179
2 -0.016990  -0.010101    0.023687
3 -0.024932  -0.000925    0.021179
4 -0.024932  -0.000925    0.021179
Means (should be ~0):
Quantity      4.701951e-19
UnitPrice    -4.119805e-19
TotalPrice   -3.349580e-18
dtype: float64
Std devs (should be ~1):
Quantity      1.000001
UnitPrice     1.000001
TotalPrice    1.000001
dtype: float64


In [210]:
# Select numeric columns you want to scale
numeric_cols = ['Quantity', 'UnitPrice', 'TotalPrice', 'CustomerID_freq', 'StockCode_freq']
numeric_cols = [c for c in numeric_cols if c in df.columns]

# --- Normalization (Min-Max scaling 0–1) ---
scaler = MinMaxScaler()
df_norm = df.copy()
df_norm[numeric_cols] = scaler.fit_transform(df_norm[numeric_cols])

# Show head
print("Normalized numeric columns head:")
print(df_norm[numeric_cols].head())

# Check min and max
print("\nMin values (should be 0):")
print(df_norm[numeric_cols].min())

print("\nMax values (should be 1):")
print(df_norm[numeric_cols].max())


Normalized numeric columns head:
   Quantity  UnitPrice  TotalPrice
0  0.500037   0.000065    0.742735
1  0.500037   0.000087    0.742728
2  0.500049   0.000071    0.742731
3  0.500037   0.000087    0.742728
4  0.500037   0.000087    0.742728

Min values (should be 0):
Quantity      0.0
UnitPrice     0.0
TotalPrice    0.0
dtype: float64

Max values (should be 1):
Quantity      1.0
UnitPrice     1.0
TotalPrice    1.0
dtype: float64


In [212]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

country_encoded = ohe.fit_transform(df[['Country']])
country_cols = ohe.get_feature_names_out(['Country'])

df_country = pd.DataFrame(country_encoded, columns=country_cols, index=df.index)
df = pd.concat([df, df_country], axis=1)

print("OHE columns:", country_cols[:10])
print(df[country_cols].head())


OHE columns: ['Country_Australia' 'Country_Austria' 'Country_Bahrain' 'Country_Belgium'
 'Country_Brazil' 'Country_Canada' 'Country_Channel Islands'
 'Country_Cyprus' 'Country_Czech Republic' 'Country_Denmark']
   Country_Australia  Country_Australia  Country_Austria  Country_Austria  \
0                0.0                0.0              0.0              0.0   
1                0.0                0.0              0.0              0.0   
2                0.0                0.0              0.0              0.0   
3                0.0                0.0              0.0              0.0   
4                0.0                0.0              0.0              0.0   

   Country_Bahrain  Country_Bahrain  Country_Belgium  Country_Belgium  \
0              0.0              0.0              0.0              0.0   
1              0.0              0.0              0.0              0.0   
2              0.0              0.0              0.0              0.0   
3              0.0              0.

In [196]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,...,Country_RSA,Country_Saudi Arabia,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-0.024932,2010-12-01 08:26:00,-0.012968,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,536365,71053,WHITE METAL LANTERN,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,-0.016990,2010-12-01 08:26:00,-0.010101,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,-0.001105,2011-12-09 12:50:00,-0.037340,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,-0.024932,2011-12-09 12:50:00,-0.019419,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,-0.032874,2011-12-09 12:50:00,0.009970,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,-0.032874,2011-12-09 12:50:00,0.009970,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [198]:
if 'InvoiceDate' in df.columns:
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
else:
    raise KeyError("InvoiceDate column missing")

In [200]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,...,Country_RSA,Country_Saudi Arabia,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-0.024932,2010-12-01 08:26:00,-0.012968,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,536365,71053,WHITE METAL LANTERN,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,-0.016990,2010-12-01 08:26:00,-0.010101,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,-0.001105,2011-12-09 12:50:00,-0.037340,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,-0.024932,2011-12-09 12:50:00,-0.019419,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,-0.032874,2011-12-09 12:50:00,0.009970,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,-0.032874,2011-12-09 12:50:00,0.009970,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [202]:
df = df.assign(TotalPrice = df['Quantity'] * df['UnitPrice'])

In [204]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,...,Country_Saudi Arabia,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-0.024932,2010-12-01 08:26:00,-0.012968,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000323
1,536365,71053,WHITE METAL LANTERN,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000023
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,-0.016990,2010-12-01 08:26:00,-0.010101,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000172
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000023
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,-0.024932,2010-12-01 08:26:00,-0.000925,17850.0,United Kingdom,2010,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,-0.001105,2011-12-09 12:50:00,-0.037340,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000041
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,-0.024932,2011-12-09 12:50:00,-0.019419,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000484
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,-0.032874,2011-12-09 12:50:00,0.009970,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000328
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,-0.032874,2011-12-09 12:50:00,0.009970,12680.0,France,2011,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000328


In [214]:

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# simple date features
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek   # Monday=0 ... Sunday=6
df['IsWeekend'] = df['DayOfWeek'].isin([5,6]).astype(int)

# verify
print("Date features sample:")
print(df[['InvoiceDate','Year','Month','DayOfWeek','IsWeekend']].head(3).to_string(index=False))


Date features sample:
        InvoiceDate  Year  Month  DayOfWeek  IsWeekend
2010-12-01 08:26:00  2010     12          2          0
2010-12-01 08:26:00  2010     12          2          0
2010-12-01 08:26:00  2010     12          2          0


In [216]:

df['CustomerID_filled'] = df['CustomerID'].fillna(0)


cust_invoice_count = df.groupby('CustomerID_filled')['InvoiceNo'].nunique()
df['CustomerInvoiceCount'] = df['CustomerID_filled'].map(cust_invoice_count)


cust_total_spend = df.groupby('CustomerID_filled')['TotalPrice'].sum()
df['CustomerTotalSpend'] = df['CustomerID_filled'].map(cust_total_spend)

# verify sample
print("Customer aggregates sample:")
print(df[['CustomerID_filled','CustomerInvoiceCount','CustomerTotalSpend']].drop_duplicates().head(5).to_string(index=False))


Customer aggregates sample:
 CustomerID_filled  CustomerInvoiceCount  CustomerTotalSpend
           17850.0                    35            4.167880
           13047.0                    18            1.104453
           12583.0                    18           -3.611609
           13748.0                     5           -0.143247
           15100.0                     6            0.011836


In [218]:
# reference date = dataset 
ref_date = df['InvoiceDate'].max()
print("Reference date:", ref_date)

# Recency = ref_date - last purchase date of customer (in days)
last_purchase = df.groupby('CustomerID_filled')['InvoiceDate'].max()
recency_days = (ref_date - last_purchase).dt.days
# Frequency and Monetary reused from above
frequency = cust_invoice_count
monetary = cust_total_spend

# create a customer-level dataframe rfm
rfm = pd.DataFrame({
    'Recency_days': recency_days,
    'Frequency': frequency,
    'Monetary': monetary
}).reset_index()

print("RFM sample (top 5 customers):")
print(rfm.head(5).to_string(index=False))


Reference date: 2011-12-09 12:50:00
RFM sample (top 5 customers):
 CustomerID_filled  Recency_days  Frequency  Monetary
           12346.0           325          2  0.098576
           12347.0             1          7  1.505541
           12348.0            74          4 -5.073302
           12349.0            18          1 -3.053646
           12350.0           309          1 -0.056886


In [ ]:
out_path = r"H:\Machine Learing\k-means clustring\retail_processed.csv"
df.to_csv(out_path, index=False)
print("Saved to:", out_path)
